---


## CARGA DE LIBRERÍA Y DATOS

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chi2, f_oneway,chi2_contingency
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cupido_IA_project/train.csv')
test = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/Cupido_IA_project/sample_submission.csv")

---

## DEFINICIÓN PREPROCESADO

---

In [ ]:
cols_to_int = ['age', 'sex', 'cp', 'restecg']

rename_dict = {
    "age": "edad",
    "sex": "sexo",
    "cp": "tipo_dolor_pecho",
    "trestbps": "tension_en_descanso",
    "chol": "colesterol",
    "fbs": "azucar",
    "restecg": "electro_en_descanso",
    "thalach": "latidos_por_minuto",
    "exang": "dolor_pecho_con_ejercicio",
    "oldpeak": "cambio_linea_corazon_ejercicio",
    "slope": "forma_linea_corazon_ejercicio",
    "ca": "num_venas_grandes",
    "thal": "estado_corazon_thal"
}

cols_a_clippear = [
    'tension_en_descanso', 'colesterol',
    'latidos_por_minuto', 'cambio_linea_corazon_ejercicio'
]

categorical_cols_to_round = [
    'num_venas_grandes', 'estado_corazon_thal', 'sexo',
    'tipo_dolor_pecho', 'dolor_pecho_con_ejercicio',
    'azucar', 'forma_linea_corazon_ejercicio', 'electro_en_descanso'
]

---

FUNCIONES DE PREPROCESADO

---

In [ ]:
def limpieza_inicial(df):
    """
    Realiza conversiones de tipos, renombres y limpieza básica de errores (-9).
    Se puede aplicar a todo el dataset antes del split.
    """
    df = df.copy()

    for col in cols_to_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    object_cols = df.select_dtypes(include=['object']).columns
    for col in object_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df.rename(columns=rename_dict)
    df.replace([-9, -9.0], np.nan, inplace=True)

    return df

In [ ]:
def limpiar_ceros_fisiologicos(X):
    X = X.copy()
    cols_imposibles_con_cero = ['tension_en_descanso', 'colesterol']
    for col in cols_imposibles_con_cero:
        if col in X.columns:
            X[col] = X[col].replace({0: np.nan, 0.0: np.nan})
    return X

In [ ]:
def clipear_outliers(X):
    X = X.copy()
    for col in cols_a_clippear:
        if col in X.columns:
            p1 = X[col].quantile(0.01)
            p99 = X[col].quantile(0.99)
            X[col] = X[col].clip(lower=p1, upper=p99)
    return X

In [ ]:
def crear_flags_mnar(df):
    """
    ACTUALIZADO (Estrategia Pruning):
    Solo creamos flag para 'num_venas_grandes'.
    'estado_corazon_thal_is_missing' se considera ruido (Grupo 1) y no se genera.
    """
    df_new = df.copy()
    # Solo venas, thal is missing se elimina por V=0.040
    cols_mnar = ['num_venas_grandes']
    for col in cols_mnar:
        if col in df_new.columns:
            df_new[f'{col}_is_missing'] = df_new[col].isna().astype(int)
    return df_new

In [ ]:
class RobustKNNImputerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.scaler = RobustScaler()
        self.imputer = KNNImputer(n_neighbors=n_neighbors, weights='distance')
        self.feature_names_in_ = None

    def fit(self, X, y=None):
        self.feature_names_in_ = X.columns if hasattr(X, 'columns') else [f"feat_{i}" for i in range(X.shape[1])]
        X_scaled = self.scaler.fit_transform(X)
        self.imputer.fit(X_scaled)
        return self

    def transform(self, X):
        X_scaled = self.scaler.transform(X)
        X_imputed_scaled = self.imputer.transform(X_scaled)
        X_imputed = self.scaler.inverse_transform(X_imputed_scaled)
        return pd.DataFrame(X_imputed, columns=self.feature_names_in_, index=X.index)

    def set_output(self, *, transform=None):
        return self

In [ ]:
def redondear_imputaciones(X):
    X = X.copy()
    for col in categorical_cols_to_round:
        if col in X.columns:
            X[col] = X[col].round()
    return X

In [ ]:
def aplicar_feature_engineering_avanzado(df):
    """
    ACTUALIZADO:
    1. Se eliminan cálculos innecesarios (rango_colesterol, porcentaje_freq).
    2. Se calculan las variables compuestas necesarias antes de borrar las fuentes.
    """
    df = df.copy()

    # 1. Flag Depresión ST (Usada en Score Stress)
    if 'cambio_linea_corazon_ejercicio' in df.columns:
        df['flag_depresion_st'] = (df['cambio_linea_corazon_ejercicio'] > 0).astype(int)

    # 2. Flag Hipertensión (Absorbe info de tension_en_descanso)
    if 'tension_en_descanso' in df.columns:
        df['flag_hipertension'] = (df['tension_en_descanso'] > 130).astype(int)

    # NOTA: Eliminado calculo de 'porcentaje_frecuencia_max' (Grupo 2 - Redundancia negativa)
    # NOTA: Eliminado calculo de 'rango_colesterol' (Grupo 1 - Ruido)

    # 3. Score Respuesta al Estrés (Driver Tier 1)
    if 'dolor_pecho_con_ejercicio' in df.columns and 'flag_depresion_st' in df.columns:
        df['score_respuesta_stress'] = df['dolor_pecho_con_ejercicio'] + df['flag_depresion_st']

    # 4. Carga de Comorbilidad (Absorbe azucar y electro)
    cols_comorbilidad = ['azucar', 'flag_hipertension', 'electro_en_descanso']
    if set(cols_comorbilidad).issubset(df.columns):
        # Nos aseguramos que electro sea binario para la suma (0 = normal, 1,2 = anormal)
        electro_punto = (df['electro_en_descanso'] > 0).astype(int)
        df['carga_comorbilidad'] = df['azucar'] + df['flag_hipertension'] + electro_punto

    return df

In [ ]:
def ejecutar_pruning_agresivo(df):
    """
    Elimina las variables de Grupo 1 (Ruido) y Grupo 2 (Redundancia/Absorbidas)
    después de haberlas utilizado para crear las variables compuestas.
    """
    df = df.copy()

    vars_a_eliminar = [
        # GRUPO 1: Ruido Puro
        'electro_en_descanso', # Usado en carga, adios.
        'colesterol',          # Inutil en este dataset, adios.
        # 'rango_colesterol' y 'thal_missing' ya no se generan.

        # GRUPO 2: Redundantes / Absorbidas
        'azucar',               # Absorbida en carga, adios.
        'tension_en_descanso'   # Absorbida en flag_hipertension, adios.
    ]

    # Eliminamos solo si existen (para evitar errores)
    cols_existentes = [c for c in vars_a_eliminar if c in df.columns]
    if cols_existentes:
        df = df.drop(columns=cols_existentes)

    return df

In [ ]:
def optimizar_k_knn(X_train, k_range=[3, 5, 7, 9, 11, 15]):
    # Versión simplificada de la original
    X_temp = limpiar_ceros_fisiologicos(X_train)
    X_temp = clipear_outliers(X_temp)
    X_complete = X_temp.dropna().copy()
    if len(X_complete) < 50: return 5

    rmse_scores = {}
    scaler = RobustScaler()
    X_scaled_array = scaler.fit_transform(X_complete)
    np.random.seed(42)
    mask = np.random.rand(*X_scaled_array.shape) < 0.1
    X_missing_sim = X_scaled_array.copy()
    X_missing_sim[mask] = np.nan

    for k in k_range:
        imputer = KNNImputer(n_neighbors=k, weights='distance')
        X_imputed = imputer.fit_transform(X_missing_sim)
        error = np.sqrt(mean_squared_error(X_scaled_array[mask], X_imputed[mask]))
        rmse_scores[k] = error

    return min(rmse_scores, key=rmse_scores.get)

---
## APLICACIÓN DEL FLUJO DE PREPROCESADO DEFINIDO

---

In [ ]:
df_train = df.copy()
df_test = test.copy()

df_train = limpieza_inicial(df_train)
df_test = limpieza_inicial(df_test)

target = "label"

X_train = df_train.drop(columns=target)
y_train = df_train[target]

if target in df_test.columns:
    X_test = df_test.drop(columns=target)
    y_test = df_test[target]
else:
    X_test = df_test.copy()

best_k = optimizar_k_knn(X_train)

# 3. Definición del Pipeline
pipeline_feature_engineering = Pipeline([
    ('limpieza_ceros', FunctionTransformer(limpiar_ceros_fisiologicos, validate=False)),
    ('clipear_outliers', FunctionTransformer(clipear_outliers, validate=False)),
    ('mnar_flags', FunctionTransformer(crear_flags_mnar, validate=False)),
    ('imputacion_robusta', RobustKNNImputerWrapper(n_neighbors=best_k)), # Usamos el K optimizado
    ('rounding', FunctionTransformer(redondear_imputaciones, validate=False)),
    ('feature_engineering', FunctionTransformer(aplicar_feature_engineering_avanzado, validate=False)),
    # NUEVO PASO: Pruning
    ('pruning_agresivo', FunctionTransformer(ejecutar_pruning_agresivo, validate=False)),
    ('final_scaler', RobustScaler())
]).set_output(transform="pandas")

---
## EJECUCIÓN Y VERIFICACIÓN DE TRANSFORMACIÓN

---

In [ ]:
print("Ajustando pipeline con estrategia de Pruning Agresivo...")
X_train_prep = pipeline_feature_engineering.fit_transform(X_train)
y_train_prep = y_train.copy()

X_test_prep = pipeline_feature_engineering.transform(X_test)
print("\n--- Proceso finalizado ---")
print(f"Dimensiones Train final: {X_train_prep.shape}")
print(f"Columnas finales en el dataset: \n{X_train_prep.columns.tolist()}")

Ajustando pipeline con estrategia de Pruning Agresivo...

--- Proceso finalizado ---
Dimensiones Train final: (732, 14)
Columnas finales en el dataset: 
['edad', 'sexo', 'tipo_dolor_pecho', 'latidos_por_minuto', 'dolor_pecho_con_ejercicio', 'cambio_linea_corazon_ejercicio', 'forma_linea_corazon_ejercicio', 'num_venas_grandes', 'estado_corazon_thal', 'num_venas_grandes_is_missing', 'flag_depresion_st', 'flag_hipertension', 'score_respuesta_stress', 'carga_comorbilidad']


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST CON SVM

---

In [ ]:
print("Entrenando SVM con el dataset PREPROCESADO...")
svm_final = SVC(probability=True, random_state=42)

svm_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = svm_final.predict(X_test_prep)


# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_svm_pruned_1.csv", index=False)
    print("¡Archivo 'submission_svm_pruned_1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

Entrenando SVM con el dataset PREPROCESADO...
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_svm_pruned_1.csv' guardado con éxito!
   ID  label
0   0      2
1   1      0
2   2      0
3   3      2
4   4      0


---
APLICACIÓN DE HIPERPARÁMETROS ÓPTIMOS SVM

---

In [ ]:
# 0.56521
print("Entrenando SVM con hiperparámetros OPTIMIZADOS (C=1, gamma=0.05)...")

svm_final = SVC(
    C=1,
    gamma=0.05,
    kernel='rbf',
    class_weight=None,
    probability=True,
    random_state=42)

svm_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = svm_final.predict(X_test_prep)


# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_svm_pruned_opt_1.csv", index=False)
    print("¡Archivo 'submission_svm_pruned_opt_1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

Entrenando SVM con hiperparámetros OPTIMIZADOS (C=1, gamma=0.05)...
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_svm_pruned_opt_1.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


2º APLICACIÓN DE HIPERPARÁMETROS ÓPTIMOS SVM (empeora la anterior en kaggle)

In [ ]:
# 0.54347
print("--- Entrenando SVM REFINADO (C=1.1, gamma=0.04) ---")

# Configuración ganadora del último GridSearch
svm_refined = SVC(
    C=1.1,
    gamma=0.04,
    kernel='rbf',
    probability=True,
    random_state=42
)

svm_refined.fit(X_train_prep, y_train)

print("Generando predicciones finales...")
y_test_refined = svm_refined.predict(X_test_prep)

if len(y_test_refined) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_svm_pruned_opt_2.csv", index=False)
    print("¡Archivo 'submission_svm_pruned_opt_2.csv' guardado con éxito!")
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden. Revisa si se borraron filas en el test.")

--- Entrenando SVM REFINADO (C=1.1, gamma=0.04) ---
Generando predicciones finales...
¡Archivo 'submission_svm_pruned_opt_2.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


---
## ENTRENAMIENTO Y PREDICCIÓN SOBRE TEST CON REGRESIÓN LOGÍSTICA

---

In [ ]:

print("Entrenando Regresión Logística con el dataset PREPROCESADO...")

lr_final = LogisticRegression(random_state=42, max_iter=1000,class_weight='balanced')
lr_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = lr_final.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    # Nombre de archivo actualizado
    submission.to_csv("submission_logreg_pruned_1.csv", index=False)
    print("¡Archivo 'submission_logreg_pruned_1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

Entrenando Regresión Logística con el dataset PREPROCESADO...
Generando predicciones sobre el Test...


---
APLICACIÓN DE HIPERPARÁMETROS ÓPTIMOS REGRESIÓN LOGÍSTICA

---

In [ ]:
#  0.54347
print("Entrenando Regresión Logística con hiperparámetros OPTIMIZADOS")

lr_final = LogisticRegression(
    C=0.08,
    solver='lbfgs',
    class_weight=None,
    random_state=42,
    max_iter=1000
)

lr_final.fit(X_train_prep, y_train)

print("Generando predicciones sobre el Test...")
y_test_pred = lr_final.predict(X_test_prep)

# Verificaciones de seguridad
print(f"Predicciones generadas: {len(y_test_pred)}")
print(f"Filas en sample_submission: {len(submission)}")

if len(y_test_pred) == len(submission):
    submission["label"] = y_test_pred
    submission.to_csv("submission_logreg_pruned_opt_1.csv", index=False)
    print("¡Archivo 'submission_logreg_pruned_opt_1.csv' guardado con éxito!")

    # Vista previa
    print(submission.head())
else:
    print("¡ALERTA! Las dimensiones no coinciden.")

Entrenando Regresión Logística con hiperparámetros OPTIMIZADOS
Generando predicciones sobre el Test...
Predicciones generadas: 184
Filas en sample_submission: 184
¡Archivo 'submission_logreg_pruned_opt_1.csv' guardado con éxito!
   ID  label
0   0      3
1   1      0
2   2      0
3   3      2
4   4      0


---
## DOCUMENTACIÓN

---

El objetivo fue superar el estancamiento del 0.559 (Iteración 2) eliminando el ruido que confundía a los modelos geométricos.

### 1. 🧪 Innovación: Pipeline de "Pruning Agresivo"
Se implementó una selección quirúrgica de variables en lugar de usar el dataset completo:
* **Ingeniería de Características:** Creación de variables sintéticas clínicas (`score_respuesta_stress`, `carga_comorbilidad`).
* **Pruning (Poda):** Eliminación de variables originales ruidosas tras extraer su valor (`colesterol`, `azucar`, etc.).
* **Optimización:** Ajuste fino del SVM (`C=1`, `gamma=0.05`) al nuevo espacio dimensional.

### 2. 🏆 Resultados en Kaggle (Nuevo Techo de Cristal)
* **🥇 SVM Pruned Optimized:** **Score 0.56521**.
    * **Hito:** Se rompe la barrera del 0.56. Este modelo establece el **máximo rendimiento obtenido en todo el proyecto hasta ahora**.
    * *Insight:* Menos es más. Al eliminar variables irrelevantes, el SVM encontró un hiperplano más generalizable que con el dataset completo.
* **🥈 SVM Pruned Base:** **0.548**. Sin ajuste de hiperparámetros, el pruning por sí solo no basta; requiere re-sintonización.
* **🥉 Regresión Logística:** **0.543**. Se mantiene estable, confirmando que ya alcanzó su límite lineal.

### 3. 📉 Conclusión
El **SVM con Pruning y Optimización** es el modelo a batir. La limpieza de variables (Feature Selection) aportó más valor que la limpieza de datos de la iteración anterior.